# Data Exploration for the Protein Dataset

In [13]:
import numpy as np
import pandas as pd
import os

from torchvision.datasets.utils import download_url
from sklearn.preprocessing import StandardScaler

## `Protein` dataset from `SSDKL` paper

In [14]:

def load_dataset_from_ssdkl(path_to_folder):
    X = np.load(os.path.join(path_to_folder, "X.npy"))
    y = np.load(os.path.join(path_to_folder, "y.npy"))

    return X, y

In [120]:
PATH_TO_DATASET = "/home/flo/ssdgm/notebooks/datasets/SSDKL/protein"
X_ssdkl, y_ssdkl = load_dataset_from_ssdkl(PATH_TO_DATASET)
X_ssdkl.shape, y_ssdkl.shape

((45730, 9), (45730,))

## `Protein` dataset from `UCI`

In [18]:
URL = "https://archive.ics.uci.edu/ml/machine-learning-databases/00265/CASP.csv"
DOWNLOAD_FOLDER = "/home/flo/ssdgm/notebooks/datasets/UCI/Protein/raw"

download_url(url=URL, root=DOWNLOAD_FOLDER)

Using downloaded and verified file: /home/flo/ssdgm/notebooks/datasets/UCI/Protein/raw/CASP.csv


In [121]:
df_uci = pd.read_csv("/home/flo/ssdgm/notebooks/datasets/UCI/Protein/raw/CASP.csv")
df_uci.shape

(45730, 10)

In [21]:
df_uci.head()

,RMSD,F1,F2,F3,F4,F5,F6,F7,F8,F9
0,17.284,13558.30,4305.35,0.31754,162.1730,1.872791e+06,215.3590,4287.87,102,27.0302
1,6.021,6191.96,1623.16,0.26213,53.3894,8.034467e+05,87.2024,3328.91,39,38.5468
2,9.275,7725.98,1726.28,0.22343,67.2887,1.075648e+06,81.7913,2981.04,29,38.8119
3,15.851,8424.58,2368.25,0.28111,67.8325,1.210472e+06,109.4390,3248.22,70,39.0651
4,7.962,7460.84,1736.94,0.23280,52.4123,1.021020e+06,94.5234,2814.42,41,39.9147


## Conclusion

- use `RMSD` as target
- no cleaning steps necessary